<a href="https://colab.research.google.com/github/JMML2021/0401_Registre-de-logs/blob/main/0401_Acceso_Expresiones_Regulares.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# importing libraries
from urllib.request import urlretrieve
import pandas as pd
import os
import re
from datetime import datetime
import pytz
#import geocoder
import folium
import time
import urllib.request
import json
#import geopandas
import matplotlib.pyplot as plt
from datetime import datetime

In [2]:
# Activo Google Drive

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Abro archivo

df = list(open('/content/drive/MyDrive/Web_access_log-akumenius.com.txt'))
file= '/content/drive/MyDrive/Web_access_log-akumenius.com.txt'
df[200:210]

['www.akumenius.com 180.76.5.31 - - [23/Feb/2014:03:15:03 +0100] "GET /destinos-baratos/destinos-caracteristicas/hoteles-baratos-en-Zurich_SUIZA-con-Alquiler-de-silla-de-ruedas HTTP/1.1" 200 8360 "-" "Mozilla/5.0 (compatible; Baiduspider/2.0; +http://www.baidu.com/search/spider.html)" VLOG=-\n',
 'www.akumenius.com 66.249.76.216 - - [23/Feb/2014:03:15:03 +0100] "GET /hoteles-baratos/ofertas-hotel-Capitolio-Apartamentos-Turisticos-en-Merida-207271b-destinos.html HTTP/1.1" 404 3100 "-" "Mozilla/5.0 (compatible; Googlebot/2.1; +http://www.google.com/bot.html)" VLOG=-\n',
 'www.akumenius.com 66.249.76.216 - - [23/Feb/2014:03:15:06 +0100] "GET /destinos-baratos/destinosEstrelles/hoteles-en-Gary_ESTADOS%20UNIDOS-con-4-estrellas HTTP/1.1" 200 8812 "-" "Mozilla/5.0 (compatible; Googlebot/2.1; +http://www.google.com/bot.html)" VLOG=-\n',
 'www.akumenius.com 66.249.76.216 - - [23/Feb/2014:03:15:06 +0100] "GET /hoteles-baratos/ofertas-hotel-Guitart-Termes-La-Collada-en-La%20Molina-6833b-destinos.

Una vez que se imprimen las líneas, vemos que están en las líneas muestra Formato de registro común (CLF):

mira aquí lo que significa CLF

sabiendo esto, mostremos la estructura del formato de registro:

%h %l %u %t "%r" %>s %b "%{Referente}i" "%{Usuario-agente}i\

dónde:

%h es la dirección IP del cliente (host remoto);

%l es la identidad RFC 1413 (no determinado);

%u es la identificación del usuario, determinada por la autenticación HTTP;

%t es la hora, por defecto, formateada como [día/mes/año:hora:minuto:segunda zona];

"%r" es la cadena de solicitud, formateada como "protocolo de recurso de método"

%>s es el código de estado;

%O es el tamaño de la solicitud;

"%{Referer}i" es el encabezado de solicitud HTTP de referencia y

"%{User-Agent}i" es el encabezado de solicitud HTTP de User-Agent.


In [15]:
def analizar_palabra(x):
    """
    Returns the string delimited by two characters.

    Example:
        `>>> parse_str('[string]')`
        `'string'`
    """
    return x[1:-1]

def analizar_fechaHora(x):
    '''
    Parses datetime with timezone formatted as:
        `[day/month/year:hour:minute:second zone]`

    Example:
        `>>> parse_datetime('13/Nov/2015:11:45:42 +0000')`
        `datetime.datetime(2015, 11, 3, 11, 45, 4, tzinfo=<UTC>)`

    Due to problems parsing the timezone (`%z`) with `datetime.strptime`, the
    timezone will be obtained using the `pytz` library.
    '''
    dt = datetime.strptime(x[1:-7], '%d/%b/%Y:%H:%M:%S')
    dt_tz = int(x[-6:-3])*60+int(x[-3:-1])
    return dt.replace(tzinfo=pytz.FixedOffset(dt_tz))

In [16]:
data = pd.read_csv(file,

    sep=r'\s(?=(?:[^"]*"[^"]*")*[^"]*$)(?![^\[]*\])',
    engine='python',
    na_values='-',
    header=None,
    usecols=[0, 1, 4, 5, 6, 7, 8, 9],
    names=['virtual_host','ip', 'time', 'request', 'status', 'size', 'referer', 'user_agent'],
    converters={'time': analizar_fechaHora,
                'request': analizar_palabra,
                'status': int,
                'size': int,
                'referer': analizar_palabra,
                'user_agent': analizar_palabra})

In [11]:
data

,virtual_host,ip,time,request,status,size,referer,user_agent
0,localhost,127.0.0.1,2014-02-23 03:10:31+01:00,OPTIONS * HTTP/1.0,200,NaN,NaN,Apache (internal dummy connection)
1,localhost,127.0.0.1,2014-02-23 03:10:31+01:00,OPTIONS * HTTP/1.0,200,NaN,NaN,Apache (internal dummy connection)
2,localhost,127.0.0.1,2014-02-23 03:10:31+01:00,OPTIONS * HTTP/1.0,200,NaN,NaN,Apache (internal dummy connection)
3,localhost,127.0.0.1,2014-02-23 03:10:31+01:00,OPTIONS * HTTP/1.0,200,NaN,NaN,Apache (internal dummy connection)
4,localhost,127.0.0.1,2014-02-23 03:10:31+01:00,OPTIONS * HTTP/1.0,200,NaN,NaN,Apache (internal dummy connection)
...,...,...,...,...,...,...,...,...
261868,www.akumenius.com,5.255.253.53,2014-03-02 03:05:39+01:00,GET / HTTP/1.1,200,7528,NaN,Mozilla/5.0 (compatible; YandexBot/3.0; +http:...
261869,www.akumenius.com,74.86.158.107,2014-03-02 03:09:52+01:00,HEAD / HTTP/1.1,200,NaN,NaN,Mozilla/5.0+(compatible; UptimeRobot/2.0; http...
261870,localhost,127.0.0.1,2014-03-02 03:10:18+01:00,OPTIONS * HTTP/1.0,200,NaN,NaN,Apache (internal dummy connection)
261871,localhost,127.0.0.1,2014-03-02 03:10:18+01:00,OPTIONS * HTTP/1.0,200,NaN,NaN,Apache (internal dummy connection)


In [ ]:
data.sample(10)